In [ ]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


In [ ]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [ ]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [ ]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [ ]:
print(df_train.shape,df_test.shape)

In [ ]:
df_train.head()

In [ ]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [ ]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [ ]:
valid_filenames

In [ ]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [ ]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [ ]:
print(df_train.shape,df_test.shape)

### Upsample minority classes

In [ ]:
df_train['genre_id'].value_counts()

In [ ]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [ ]:
df_train.tail()#,df_train.shape

In [ ]:
df_train['genre_id'].value_counts()

### Create Folds

In [ ]:
df_train['fold'] = 0

In [ ]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [ ]:
df_train.head()

In [ ]:
df_test.head()

#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
#     [torchvision.transforms.Resize((128,128))])
    [torchvision.transforms.CenterCrop((128,960))])


In [ ]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
#                      tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds',mode="per_example",p_mode="per_example",sample_rate=16000),
#                     tA.BandPassFilter(),
                    tA.TimeInversion(p=0.5,p_mode="per_example",sample_rate=16000),
#                     tA.PitchShift()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.5,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [ ]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 64,
                 PL=True
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        self.PL = PL
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if not self.PL:
            if self.train:
                audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/{filename}')
            else:
                audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/{filename}')
        else:
            try:
                audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/{filename}')
            except:
                audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/{filename}')
                
        audio = np.mean(audio, axis=0, keepdims=True)   ### convert to mono
        return audio

    def resample(self,audio,sr):
        
        audio = torchaudio.transforms.Resample(orig_freq= sr, new_freq=  16000, 
                                    resampling_method = 'sinc_interpolation', 
                                    lowpass_filter_width = 6, 
                                    rolloff = 0.99)(audio)
        return audio

    def makeCQT(self,audio,sr):
        audio = CQT1992v2(hop_length=self.HOP_LEN,n_bins=self.n_bins,bins_per_octave=self.bins_per_octave,
                          sr=sr,verbose=False)(audio)
        
        max_ = audio.max()
        if max_ < 0.001:
            audio[...] = 0
        else:
            audio /= max_
        
        return audio
        
        
    def create_spectrogram(self,audio,sr):
                
        specgram = torchaudio.transforms.MelSpectrogram(sample_rate=sr,
                                                        n_fft=self.N_FFT, 
                                                        win_length=self.N_FFT, 
                                                        hop_length=self.HOP_LEN,
                                                        center=True,
                                                        pad_mode="reflect",
                                                        power=2.0,
                                                        norm='slaney',
                                                        onesided=True,
                                                        n_mels=self.n_mels,
                                                        mel_scale="htk"
                                                       )(audio)

        specgram = torchaudio.transforms.AmplitudeToDB()(specgram)
        specgram = specgram - specgram.min()
        
        max_ = specgram.max()
        if max_ < 0.001:
            specgram[...] = 0
        else:
            specgram /= max_
        return specgram

    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        sr = 16000
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
#         samples = self.load_file(row.filename)
    
        samples = np.stack(samples, axis=0)
        
        
        samples = torch.from_numpy(samples).float().view(1,-1)
        
        if self.audiotfms is not None:
             samples = self.audiotfms(samples.unsqueeze(0)).squeeze(0)

        samples = self.create_spectrogram(samples,sr)
#         samples = self.makeCQT(samples,sr)

        if self.transform is not None:
            samples = self.transform(samples)
            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [ ]:
# a,b = next(iter(POGFiles(df_train)))
# a.shape

In [ ]:
a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=448,N_FFT=512)))
#` a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=576,N_FFT=512)))
a.shape

In [ ]:
def gem_1d_res(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), 2,2).pow(1./p)

def gem_1d(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), (x.size(-1),)).pow(1./p )

#### FastAI model

In [ ]:
df_train['genre_id'].nunique()

In [ ]:
df_train.head()

In [ ]:
import timm
timm.list_models()

In [ ]:
timm.create_model('resnest50d',in_chans=1)

In [ ]:
df_train['genre_id'].value_counts()

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
PL = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification\outputs\resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128\PL.csv').drop('Unnamed: 0',1)
PL['genre_id'] = np.argmax(PL.iloc[:,:-1].values,1)
PL = PL.iloc[:,-2:]
PL = pd.merge(PL,df_test,on='song_id',how='left')
PL['fold'] = df_train['fold'].max()+1
PL.head()

In [ ]:
df_train.drop('genre',1,inplace=True)
df_train.head()

In [ ]:
PL = PL[df_train.columns]
PL.head()

In [ ]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'resnet50d_PL_finetune_MonoSpec_Hop512_Mels128'
        kernel_type_prev = 'resnest50d_MonoSpec_Hop512_Mels128'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 32
        n_epochs = 75

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        training_fold  = pd.concat([training_fold,PL],0).reset_index(drop=True)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms,
                            n_mels=128,HOP_LEN=512,N_FFT=512
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,
                            n_mels=128,HOP_LEN=512,N_FFT=512
                            )

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('resnest50d', pretrained=False, in_chans=1)#get_model(resnet34)
        model.conv1[0].stride = (1,1)
        model.fc = nn.Linear(2048,19)

        f1_score = F1Score(average="micro")
        
        
        learn = Learner(dls, model,opt_func=ranger,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
        learn.fit_flat_cos(n_epochs, 1e-2, wd=1e-03)
#         learn = learn.to_fp16()
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           n_mels=128,HOP_LEN=512,N_FFT=512,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [ ]:
if __name__=='__main__':
    run()

In [ ]:
submission

### OOF

In [ ]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [ ]:
OOFOut['ID']

In [ ]:
OOFOut.tail()

In [ ]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [ ]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [ ]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

In [ ]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

In [ ]:
df = OOFOut.groupby('genre_id').size()
df = 1. / df
df = df / df.mean()

class_weights = df.values
logits_weights = np.log((class_weights).reshape((1, -1)))
probit_weights = scipy.special.softmax(logits_weights,1)

In [ ]:
probit_weights

In [ ]:
f1_score(OOFOut['genre_id'],OOFOut.iloc[:,:19].idxmax(1),average="micro")

In [ ]:
for a in [13,14,15,16]:
    print(a, OOFOut[OOFOut['genre_id'].isin([a])][a].mean(),OOFOut[~OOFOut['genre_id'].isin([a])][a].mean())

### Mode based sub

In [ ]:
kernel_type

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                    'resnest50d_MonoSpec_Hop512_Mels128',
                    ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2bestmodels.csv',index=False)

### Mode based sub: specific folds only

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                    'resnest50d_MonoSpec_Hop512_Mels128',
                    ]:
        if kernels == 'resnest50d_MonoSpec_Hop512_Mels128':
            if fld in [0,1,2]:
                fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
                out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
        else:
            fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
            out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')

submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2bestmodels_bestfoldsonly.csv',index=False)

### OOF analysis

In [ ]:
OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/oof_fold_{fld}_probs.npy',allow_pickle=True)

OOF = ([item for sublist in OOF for item in sublist])
OOF = pd.DataFrame(OOF)

In [ ]:
OOFOut.shape

In [ ]:
OOFOut

In [ ]:
kernel_type_upd = 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128' 

In [ ]:
fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/fold_{fld}.csv')
fname

In [ ]:
preds_out = pd.DataFrame(np.zeros((5076,19)))
for fld in range(5):
    probs = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/test_fold_{fld}_probs.npy',allow_pickle=True)
    p = pd.DataFrame([item.numpy().ravel() for sublist in probs for item in sublist])
    preds_out += p.values/5

In [ ]:
preds_out.shape

In [ ]:
preds_out['song_id'] = fname['song_id']

In [ ]:
preds_out.head()

In [ ]:
preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0]

In [ ]:
PL = preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0].reset_index(drop=True)

In [ ]:
PL.to_csv(r'C:\Users\Kaggle\Pog_Music_Classification\outputs\resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128\PL.csv')

In [ ]:
PL.head()

In [ ]:
np.argmax(PL.iloc[:,:-1].values,1)

### End ###